In [ ]:
# 1) Cell 1: Libraries, Project Design Input Parameters, and Calling Metocean Data - Weather Windows
import pandas as pd
import numpy as np
from datetime import timedelta

# variables for input parameters and file paths
GCR = None  # Great Circle Route distance in km
ACL = None  # Array Cable Longitude distance in km
P = None    # Total number of FOWTs
NSS = None # Total number of Substations 
PORT_WEATHER_DATA_PATH = '' # Insert weather window data file directory for Staging & Integration Port
WEA_WEATHER_DATA_PATH = '' # Insert weather window data directory for WEA

# input parameters
def get_input_parameters():
    global GCR, ACL, P, NSS
    GCR = float(input("Enter the Great Circle Route distance (in km): "))
    ACL = float(input("Enter the Array Cable Longitude distance (in km): "))
    P = int(input("Enter the total number of FOWTs: "))
    NSS = int(input("Enter the total number of Substations: "))

# weather window data from Excel files
def load_weather_data(file_path):
    return pd.read_excel(file_path)

# get input parameters
get_input_parameters()

# load weather data for Staging & Integration Port and WEA (Wind Energy Area)
port_weather_data = load_weather_data(PORT_WEATHER_DATA_PATH)
wea_weather_data = load_weather_data(WEA_WEATHER_DATA_PATH)

print(f"GCR: {GCR}, ACL: {ACL}, P: {P}, NSS: {NSS}")

In [ ]:
# 2) Task Info Cell (Base Case): Defining task information, dependencies, constraints, and number of repetitions for each task.
tasks_info = {
    'Step 1': {
        'duration': 'GCR / 4',  # to be calculated with GCR input
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3'], # weather windows allowed for this task to be executed
        'repetitions': '1', # number of repititions this task will be executed for the complete project
        'location': 'WEA', # location of the task (port or WEA) for weather window selection
    },
    'Step 2': {
        'duration': '48',
        'weather_conditions': ['WC_1', 'WC_2'],
        'repetitions': 'NSS', 
        'location': 'port',  
    },
    'Step 3': {
        'duration': '36', #updated for case 4, longer integration time although it's assummed that it will be shorther by the time FOWT installations being conducted for commercial scale projects.
        'weather_conditions': ['WC_1', 'WC_2'],
        'repetitions': 'P',  
        'location': 'port', 
    },
    'Step 4': {
        'duration': '8 + (GCR / 22)',
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3', 'WC_4'],
        'repetitions': 'P',
        'location': 'WEA', 
    },
    'Step 5': {
        'duration': 'ACL / 4',  
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3'],
        'repetitions': '1',
        'location': 'WEA', 
    },
    'Step 6': {
        'duration': '18', 
        'weather_conditions': ['WC_1', 'WC_2'],
        'repetitions': 'P',
        'location': 'port'
    },
    'Step 7': {
        'duration': '18', 
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3'],
        'repetitions': 'P',
        'location': 'WEA'
    },
    'Step 8': {
        'duration': 'GCR / 4.5',
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3', 'WC_4'], 
        'repetitions': 'NSS', 
        'location': 'WEA', 
    },
    'Step 9': {
        'duration': 'GCR / 7.5', 
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3', 'WC_4'], 
        'repetitions': 'P/2', 
        'location': 'WEA', 
    },
    'Step 10': {
        'duration': '24',
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3'],
        'repetitions': 'NSS', 
        'location': 'WEA', 
    },
    'Step 11': {
        'duration': '36',  
        'dependencies': ['Step 9'],  
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3'],
        'start_time': None,
        'repetitions': 'P/2',  
        'location': 'WEA', 
    },
    'Step 12': {
        'duration': '5 * GCR', # may need to update, the process pace may be faster or slower depending on the vessel capabilities at the time of commercial scale deployments of FOWTs and location specifications.'
        'dependencies': ['Step 1', 'Step 10'],
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3'],
        'start_time': None,
        'repetitions': 'NSS',
        'location': 'WEA', 
    },
    'Step 13': {
        'duration': '5 * ACL', # may need to update, the process pace may be faster or slower depending on the vessel capabilities at the time of commercial scale deployments of FOWTs and location specifications.'
        'dependencies': ['Step 1', 'Step 5', 'Step 10', 'Step 11'],
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3'],
        'start_time': None,
        'repetitions': '1',
        'location': 'WEA'
    },
    'Step 14': {
        'duration': '24',
        'dependencies': ['Step 12'],
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3'],
        'start_time': None,
        'repetitions': 'NSS', 
        'location': 'WEA', 
    },
    'Step 15': {
        'duration': '12',
        'dependencies': ['Step 13'],
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3'],
        'start_time': None,
        'repetitions': 'P', 
        'location': 'WEA', 
    },
}

In [137]:
# 3) Task Info Cell (Sensitivity Case 1): Bottleneck steps' duration time and operational weather windows are updated for sensitivity analysis, (Steps 3, 6, 7, 11, 15)
tasks_info = {
    'Step 1': {
        'duration': 'GCR / 4',  # to be calculated with GCR input
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3'], # weather windows allowed for this task to be executed
        'repetitions': '1', # number of repititions this task will be executed for the complete project
        'location': 'WEA', # location of the task (port or WEA) for weather window selection
    },
    'Step 2': {
        'duration': '48',
        'weather_conditions': ['WC_1', 'WC_2'],
        'repetitions': 'NSS',
        'location': 'port',  
    },
    'Step 3': {
        'duration': '30',
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3'],
        'repetitions': 'P', 
        'location': 'port', 
    },
    'Step 4': {
        'duration': '8 + (GCR / 22)',
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3', 'WC_4'],
        'repetitions': 'P',
        'location': 'WEA', 
    },
    'Step 5': {
        'duration': 'ACL / 4',  # to be calculated with input for ACL
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3'],
        'repetitions': '1',
        'location': 'WEA', 
    },
    'Step 6': {
        'duration': '16',
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3'],
        'repetitions': 'P',
        'location': 'port'
    },
    'Step 7': {
        'duration': '16', 
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3', 'WC_4'],
        'repetitions': 'P',
        'location': 'WEA'
    },
    'Step 8': {
        'duration': 'GCR / 4.5',
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3', 'WC_4'], 
        'repetitions': 'NSS', 
        'location': 'WEA', 
    },
    'Step 9': {
        'duration': 'GCR / 7.5', 
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3', 'WC_4'], 
        'repetitions': 'P/2', 
        'location': 'WEA', 
    },
    'Step 10': {
        'duration': '24',
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3'],
        'repetitions': 'NSS', 
        'location': 'WEA', 
    },
    'Step 11': {
        'duration': '32', 
        'dependencies': ['Step 9'],  
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3', 'WC_4'],
        'start_time': None,
        'repetitions': 'P/2',  
        'location': 'WEA', 
    },
    'Step 12': {
        'duration': '5 * GCR', 
        'dependencies': ['Step 1', 'Step 10'],
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3', 'WC_4'],
        'start_time': None,
        'repetitions': 'NSS', 
        'location': 'WEA', 
    },
    'Step 13': {
        'duration': '5 * ACL', 
        'dependencies': ['Step 1', 'Step 5', 'Step 10', 'Step 11'],
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3', 'WC_4'],
        'start_time': None,
        'repetitions': '1',
        'location': 'WEA'
    },
    'Step 14': {
        'duration': '24',
        'dependencies': ['Step 12'],
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3'],
        'start_time': None,
        'repetitions': 'NSS', 
        'location': 'WEA', 
    },
    'Step 15': {
        'duration': '10',
        'dependencies': ['Step 13'],
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3', 'WC_4'],
        'start_time': None,
        'repetitions': 'P', 
        'location': 'WEA', 
    },
}

In [197]:
# 4) Task Info Cell (Sensitivity Case 2): Bottleneck steps' duration time updated for sensitivity analysis, (Steps 3, 6, 7, 11, 15).
tasks_info = {
    'Step 1': {
        'duration': 'GCR / 4',  # to be calculated with GCR input
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3'], # weather windows allowed for this task to be executed
        'repetitions': '1', # number of repititions this task will be executed for the complete project
        'location': 'WEA', # location of the task (port or WEA) for weather window selection
    },
    'Step 2': {
        'duration': '48',
        'weather_conditions': ['WC_1', 'WC_2'],
        'repetitions': 'NSS', 
        'location': 'port',  
    },
    'Step 3': {
        'duration': '30',
        'weather_conditions': ['WC_1', 'WC_2'],
        'repetitions': 'P',  
        'location': 'port', 
    },
    'Step 4': {
        'duration': '8 + (GCR / 22)',
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3', 'WC_4'],
        'repetitions': 'P',
        'location': 'WEA', 
    },
    'Step 5': {
        'duration': 'ACL / 4',  # to be calculated with input for ACL
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3'],
        'repetitions': '1',
        'location': 'WEA', 
    },
    'Step 6': {
        'duration': '16', 
        'weather_conditions': ['WC_1', 'WC_2'],
        'repetitions': 'P',
        'location': 'port'
    },
    'Step 7': {
        'duration': '16', 
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3'],
        'repetitions': 'P',
        'location': 'WEA'
    },
    'Step 8': {
        'duration': 'GCR / 4.5',
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3', 'WC_4'], 
        'repetitions': 'NSS', 
        'location': 'WEA', 
    },
    'Step 9': {
        'duration': 'GCR / 7.5', 
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3', 'WC_4'], 
        'repetitions': 'P/2', 
        'location': 'WEA', 
    },
    'Step 10': {
        'duration': '24',
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3'],
        'repetitions': 'NSS', 
        'location': 'WEA', 
    },
    'Step 11': {
        'duration': '32', 
        'dependencies': ['Step 9'],  
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3'],
        'start_time': None,
        'repetitions': 'P/2',  
        'location': 'WEA', 
    },
    'Step 12': {
        'duration': '5 * GCR', 
        'dependencies': ['Step 1', 'Step 10'],
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3'],
        'start_time': None,
        'repetitions': 'NSS', 
        'location': 'WEA', 
    },
    'Step 13': {
        'duration': '5 * ACL', 
        'dependencies': ['Step 1', 'Step 5', 'Step 10', 'Step 11'],
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3'],
        'start_time': None,
        'repetitions': '1',
        'location': 'WEA'
    },
    'Step 14': {
        'duration': '24',
        'dependencies': ['Step 12'],
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3'],
        'start_time': None,
        'repetitions': 'NSS', 
        'location': 'WEA', 
    },
    'Step 15': {
        'duration': '10',
        'dependencies': ['Step 13'],
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3'],
        'start_time': None,
        'repetitions': 'P', 
        'location': 'WEA', 
    },
}

In [ ]:
# 5) Execution Cell 3: Simulation Execution and Result Generation (Loop over 50-year)

import pandas as pd
from dateutil import parser
from datetime import timedelta 

# Function to check if weather conditions are suitable for the timebeing.
def is_weather_condition_suitable(task, start_time, port_weather_data, wea_weather_data):
    # Choose the correct weather data based on the task's location (port or wind energy area (WEA)).
    weather_data = port_weather_data if task['location'] == 'port' else wea_weather_data
    # Sort the weather data by the 'Start Time' of the weather intervals.
    sorted_intervals = weather_data.sort_values('Start Time')

    # Iterate through the sorted weather intervals.
    for index, row in sorted_intervals.iterrows():
        # If the start time of the weather interval is after the task's start time, stop checking.
        if row['Start Time'] > start_time:
            break
        # If the task's time is within a suitable weather interval, return True.
        if row['End Time'] >= start_time and row['WC_Category'] in task['weather_conditions']:
            return True
    # If no suitable interval is found, return False.
    return False

# Function to execute the simulation of tasks based on weather data and start times.
def execute_simulation(tasks_info, port_weather_data, wea_weather_data, start_time_str):
    # Dictionary to keep track of completed tasks.
    completed_tasks = {}
    # Locks for tasks based on their sequence number, if greater than 5, they start locked.
    task_locks = {task: True if int(task.split(' ')[1]) > 5 else False for task in tasks_info}

    # Initialize execution counts for steps 3, 4, 6, 7, 9, and 11 to zero.
    step_3_executions = 0
    step_4_executions = 0
    step_6_executions = 0
    step_7_executions = 0
    step_9_executed_count = 0
    step_11_executed_count = 0
    # Flags to indicate if step 2 has been executed at least once, and if steps 6 and 7 have both been executed at least once.
    step_2_executed_once = False
    step_6_7_executed_once = False

    # Parse the simulation start time from a string into a datetime and store the simulation start time for reference.
    current_time = parser.parse(start_time_str)
    simulation_start_time = current_time

    # List to collect task execution details for analysis.
    task_data = []

    # Main simulation loop that runs until all tasks are completed the required number of times.
    while not all(completed_tasks.get(task_name, 0) >= int(eval(str(task.get('repetitions', '1')), {'NSS': NSS}, {'P': P}))
               for task_name, task in tasks_info.items()):
        # Flag to check if any tasks have been executed in the current iteration.
        tasks_executed = False

        # Before checking conditions, calculate the total number of executions for steps 3, 6, 4, 7, 9, 11.
        total_executions_3_6 = step_3_executions + step_6_executions
        total_executions_4_7 = step_4_executions + step_7_executions
        total_executions_9_11 = step_9_executed_count + step_11_executed_count

        # Iterate over each task.
        for task_name, task in tasks_info.items():
            # Get the number of repetitions for the task.
            task_repetitions = int(eval(str(task.get('repetitions', '1')), {'NSS': NSS}, {'P': P}))
            # Calculate task duration.
            task_duration = eval(task['duration'], {'GCR': GCR, 'ACL': ACL, 'P': P, 'NSS': NSS})
            
            # Logic for Step 3,4,6,7
            if task_name == 'Step 3':
                if task_locks['Step 3'] or step_3_executions > step_6_executions:
                    continue

            if task_name == 'Step 6':
                if task_locks['Step 6']:
                    continue

            if task_name == 'Step 4':
                if task_locks['Step 4'] or step_4_executions > step_7_executions:
                    continue

            if task_name == 'Step 7':
                if task_locks['Step 7']:
                    continue

            # Additional logic for Step 7 and 4
            if task_name in ['Step 4', 'Step 7']:
                if total_executions_4_7 >= 8:  # After first two cycles
                    if total_executions_4_7 > (total_executions_9_11 * 4):
                        continue  # Wait for Steps 9 and 11 to catch up
            
            # Additional Logic for Step 3 and 6
            if task_name in ['Step 3', 'Step 6']:
                if total_executions_3_6 >= 8:  # After first 3 cycles
                    if total_executions_3_6 > (total_executions_9_11 * 4):
                        continue  # Wait for Steps 9 and 11 to catch up
        
            # Revised logic for Step 9 and 11
            if task_name in ['Step 9', 'Step 11']:
                # Check if total_executions_9_11 is approaching its final execution
                if total_executions_9_11 >= (P - 1):
                    # Allow last execution if it's exactly half of total_executions_3_6
                    if total_executions_9_11 >= (total_executions_3_6 // 2):
                        continue
                else:
                    # Standard check for most of the simulation
                    if total_executions_9_11 >= (total_executions_3_6 // 2) - 1:
                        continue  # Wait for another round of Steps 3,4,6,7

            # Revised logic for Step 9 and 11
            if task_name in ['Step 9', 'Step 11']:
                # Check if total_executions_9_11 is approaching its final execution
                if total_executions_9_11 >= (P - 1):
                    # Allow last execution if it's exactly half of total_executions_4_7
                    if total_executions_9_11 >= (total_executions_4_7 // 2):
                        continue
                else:
                    # Standard check for most of the simulation
                    if total_executions_9_11 >= (total_executions_4_7 // 2) - 1:
                        continue  # Wait for another round of Steps 3,4,6,7

            # Check if the task has already been completed the required number of times or is locked.
            if completed_tasks.get(task_name, 0) >= task_repetitions or task_locks[task_name]:
                continue

            # Check if weather conditions are suitable for the task including its duration.
            if is_weather_condition_suitable(task, current_time, port_weather_data, wea_weather_data, task_duration):
                task_start_time = current_time
                print(f"Executing {task_name} at {task_start_time}")

                # Update the current time by adding the task duration to it.
                current_time += timedelta(hours=task_duration)
                task_end_time = current_time

                # Update the count of completed tasks.
                completed_tasks[task_name] = completed_tasks.get(task_name, 0) + 1
                tasks_executed = True

                # Collect task details for analysis.
                task_details = {
                    'Task Name/Number': task_name,
                    'Start Date': task_start_time.strftime("%m/%d/%Y %H:%M:%S"),
                    'End Date': task_end_time.strftime("%m/%d/%Y %H:%M:%S"),
                    'Current Repetition': completed_tasks[task_name],
                    'Days Since Project Start': (task_start_time - simulation_start_time).days
                }
                task_data.append(task_details)

                # Unlocking logic for Step 13
                if 'Step 13' in task_locks and task_locks['Step 13']:
                    if step_11_executed_count >= (P / 2) and completed_tasks.get('Step 10', 0) >= 1:
                        if all(completed_tasks.get(step, 0) >= P for step in ['Step 3', 'Step 4', 'Step 6', 'Step 7']):
                            task_locks['Step 13'] = False
                            print("Step 13 unlocked")

                # Add these print statements to track the completion counts
                if task_name in ['Step 3', 'Step 4', 'Step 6', 'Step 7', 'Step 9', 'Step 10', 'Step 11']:
                    print(f"Completion Counts - Step 3: {completed_tasks.get('Step 3', 0)}, Step 4: {completed_tasks.get('Step 4', 0)}, Step 6: {completed_tasks.get('Step 6', 0)}, Step 7: {completed_tasks.get('Step 7', 0)}, Step 9: {completed_tasks.get('Step 9', 0)}, Step 10: {completed_tasks.get('Step 10', 0)}, Step 11: {completed_tasks.get('Step 11', 0)}")

                if task_name == 'Step 3':
                    step_3_executions += 1
                elif task_name == 'Step 4':
                    step_4_executions += 1
                elif task_name == 'Step 6':
                    step_6_executions += 1
                elif task_name == 'Step 7':
                    step_7_executions += 1
                elif task_name == 'Step 9':
                    step_9_executed_count += 1
                elif task_name == 'Step 11':
                    step_11_executed_count += 1
                elif task_name == 'Step 2':
                    step_2_executed_once = True

                # Update lock status based on task completion
                if task_name in ['Step 3', 'Step 4', 'Step 6', 'Step 7', 'Step 9', 'Step 11']:
                    task_locks[task_name] = True  # Lock task after execution

                # Unlocking logic for other steps
                if task_name == 'Step 3':
                    task_locks['Step 6'] = False
                elif task_name == 'Step 4':
                    task_locks['Step 7'] = False
                elif task_name in ['Step 6', 'Step 7']:
                    task_locks['Step 3'] = task_locks['Step 4'] = False
                    if step_6_executions >= 1 and step_7_executions >= 1:
                        step_6_7_executed_once = True

                # Unlocking Step 8
                if (step_2_executed_once and step_6_7_executed_once) or task_name == 'Step 8':
                    task_locks['Step 8'] = False

                if task_name == 'Step 8':
                    task_locks['Step 10'] = False

                # Step 12 unlocking and further steps
                if task_name == 'Step 10':
                    if completed_tasks.get('Step 1', 0) >= 1:
                        task_locks['Step 12'] = False
                
                # Step 12 unlocking and further steps
                if task_name == 'Step 1':
                    if completed_tasks.get('Step 10', 0) >= 1:
                        task_locks['Step 12'] = False

                if task_name == 'Step 12':
                    task_locks['Step 14'] = False

                if task_name == 'Step 9':
                    task_locks['Step 11'] = False

                if task_name == 'Step 11':
                    task_locks['Step 9'] = False

                # Unlocking logic for Step 13
                if task_name == 'Step 11' and completed_tasks.get('Step 11', 0) >= (P / 2) and completed_tasks.get('Step 10', 0) >= 1:
                    if all(completed_tasks.get(step, 0) >= P for step in ['Step 3', 'Step 4', 'Step 6', 'Step 7']):
                        task_locks['Step 13'] = False

                if task_name == 'Step 13':
                    task_locks['Step 15'] = False

                # Unlocking Step 9
                if step_6_executions >= 2 and step_7_executions >= 2 and not step_9_executed_count:
                    task_locks['Step 9'] = False

                print(f"Task {task_name} executed and finished at {task_end_time}. Lock status: {task_locks}") #Lock status: {task_locks}

        if not tasks_executed:
            current_time += timedelta(hours=1)

    # After simulation completes
    simulation_end_time = current_time
    total_duration = simulation_end_time - simulation_start_time
    print("Simulation Complete.")
    print(f"Total Simulation Duration: {total_duration.days} days, {total_duration.seconds // 3600} hours, {(total_duration.seconds // 60) % 60} minutes")

    return completed_tasks, task_data

# Define years for simulation in chunks
simulation_chunks = [(1972, 1981), (1982, 1991), (1992, 2001),(2002, 2011), (2012, 2021)] #(1972, 1981), (1982, 1991), (1992, 2001),(2002, 2011), (2012, 2021)

for chunk in simulation_chunks:
    for year in range(chunk[0], chunk[1] + 1, 2):
        print(f"Starting simulation for the year: {year}")
        start_time_str = f"{year}/01/01 00:00:00 AM"
        completed_tasks, task_data = execute_simulation(tasks_info, port_weather_data, wea_weather_data, start_time_str)
        
        df = pd.DataFrame(task_data)
        excel_path = f'\\X_Case_X\\simulation_data_{year}.xlsx' #Insert results directory for simulation data
        df.to_excel(excel_path, index=False)
        print(f"Results extracted to Excel file at {excel_path}")
        print(f"Simulation for {year} completed.")